- encoder forward
    - src_mask: 实现因果注意力（casual），这种 mask 一般用在 decoder 中；
        - 编码器需要全面地了解整个输入序列，以捕获全局特征。
    - src_key_padding_mask
        - 在处理变长序列时（整理成 batch 时），对齐批次中的序列长度所添加的填充符号进行掩盖。
        - key 指的是 QKV 中的 key，
- decoder forward
    - tgt_mask：实现因果注意力（casual）；
    - tgt_key_padding_mask
    - memory_mask
    - memory_key_padding_mask
- 底层主要是 `MultiheadAttention` 的 forward
    - attn_mask
    - key_padding_mask

### QKV

$$
\text{Attention}(Q,K,V)=\text{softmax}\left(\frac{QK^T}{\sqrt{d_k}}\right)V
$$
- QKV
    - Q: $\text{seq\_len}_Q,d_k$
    - K: $\text{seq\_len}_K,d_k$
    - V: $\text{seq\_len}_K,d_v$
    - K和V的长度是一致的；
- 查询（Query）决定了谁在“看”
    - 查询的位置对应于输出的位置，即我们希望为哪些位置生成新的表示。
    - 通常，我们不会掩盖查询的位置，因为这意味着不为该位置生成表示。
- 值（Value）决定了提供什么信息
    - 虽然值和键的序列长度相同，但在注意力计算中，掩盖键的位置已经足够，因为对应的值也会被忽略。
- 键（Key）决定了能“被看见”哪些信息：通过掩盖键的位置，我们控制了查询能够关注哪些位置。
    - 在注意力计算中，每个查询位置 $i$ 会对所有键位置 $j$ 计算注意力得分 $\text{Scores}_{i,j}$。
    - 当我们想要掩盖键的位置（如填充的位置），防止查询关注到这些位置，就需要对键应用掩码。
    - `key_padding_mask`: $\text{batch\_size}, \text{seq\_len}_K$
        - 对于批次中的每个样本，标记哪些键位置是需要被掩盖的。

### `mask` 与 `key_padding_mask`

In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F

In [2]:
# 定义一些超参数
batch_size = 2
seq_len = 4
d_model = 8  # 嵌入维度

In [3]:
src_mask = torch.triu(torch.ones(seq_len, seq_len), diagonal=1).bool()
src_mask

tensor([[False,  True,  True,  True],
        [False, False,  True,  True],
        [False, False, False,  True],
        [False, False, False, False]])

In [4]:
# 序列1：长度为4
seq1 = torch.tensor([1, 2, 3, 4])

# 序列2：长度为2，需要填充
seq2 = torch.tensor([5, 6])

In [5]:
pad_token = 0
padded_seq2 = torch.nn.functional.pad(seq2, (0, seq_len - len(seq2)), value=pad_token)
padded_seq2

tensor([5, 6, 0, 0])

In [6]:
src = torch.stack([seq1, padded_seq2])  # 形状：[batch_size, seq_len]
src

tensor([[1, 2, 3, 4],
        [5, 6, 0, 0]])

In [7]:
src_key_padding_mask = (src == pad_token)  # 形状：[batch_size, seq_len]
src_key_padding_mask

tensor([[False, False, False, False],
        [False, False,  True,  True]])

### MHA

In [8]:
# 定义输入参数
batch_size = 2
seq_len = 4
embed_dim = 3
num_heads = 1  # 为了简单起见，使用单头注意力

# 创建输入张量（查询、键、值），形状为 [seq_len, batch_size, embed_dim]
# 注意：在 PyTorch 中，注意力模块的输入形状是 [seq_len, batch_size, embed_dim]
query = torch.randn(seq_len, batch_size, embed_dim)
key = torch.randn(seq_len, batch_size, embed_dim)
value = torch.randn(seq_len, batch_size, embed_dim)

In [9]:
src_mask = torch.triu(torch.ones(seq_len, seq_len), diagonal=1).bool()
src_mask

tensor([[False,  True,  True,  True],
        [False, False,  True,  True],
        [False, False, False,  True],
        [False, False, False, False]])

In [10]:
src_key_padding_mask = torch.tensor([
    [False, False, False, True],  # 第一个序列，最后一个位置是填充
    [False, False, False, False]  # 第二个序列，无填充
])

In [11]:
# 定义线性变换的权重和偏置（为了简单，我们使用随机初始化）
W_q = torch.randn(embed_dim, embed_dim)
W_k = torch.randn(embed_dim, embed_dim)
W_v = torch.randn(embed_dim, embed_dim)

# 偏置项
b_q = torch.randn(embed_dim)
b_k = torch.randn(embed_dim)
b_v = torch.randn(embed_dim)

In [12]:
# 定义 out_proj 的权重和偏置
W_o = torch.eye(embed_dim)  # 使用单位矩阵
b_o = torch.zeros(embed_dim)

In [13]:
# 设置 in_proj_weight 和 in_proj_bias
in_proj_weight = torch.cat([W_q, W_k, W_v], dim=0)  # [3 * embed_dim, embed_dim]
in_proj_bias = torch.cat([b_q, b_k, b_v])  # [3 * embed_dim]

In [14]:
# 转置输入以匹配线性层的输入形状 [batch_size, seq_len, embed_dim]
query_t = query.transpose(0, 1)  # [batch_size, seq_len, embed_dim]
key_t = key.transpose(0, 1)
value_t = value.transpose(0, 1)

In [15]:
# 进行线性变换
Q = torch.matmul(query_t, W_q.T) + b_q  # [batch_size, seq_len, embed_dim]
K = torch.matmul(key_t, W_k.T) + b_k
V = torch.matmul(value_t, W_v.T) + b_v

In [16]:
scores = torch.bmm(Q, K.transpose(1, 2)) / (embed_dim ** 0.5)  # [batch_size, seq_len, seq_len]

In [17]:
# 应用 src_mask
src_mask_expanded = src_mask.unsqueeze(0).expand(batch_size, seq_len, seq_len)
scores = scores.masked_fill(src_mask_expanded, float('-inf'))

In [18]:
# 应用 src_key_padding_mask，需要在注意力权重矩阵中屏蔽对应的键的位置
key_padding_mask_expanded = src_key_padding_mask.unsqueeze(1).expand(batch_size, seq_len, seq_len)
scores = scores.masked_fill(key_padding_mask_expanded, float('-inf'))

In [19]:
# 计算注意力权重
attn_weights = F.softmax(scores, dim=-1)  # [batch_size, seq_len, seq_len]

# 计算注意力输出
attn_output = torch.bmm(attn_weights, V)  # [batch_size, seq_len, embed_dim]

# 应用 out_proj 层
output = torch.matmul(attn_output, W_o.T) + b_o  # [batch_size, seq_len, embed_dim]

# 转置回 [seq_len, batch_size, embed_dim]
output = output.transpose(0, 1)

In [20]:
output

tensor([[[ 0.1002, -1.2877, -2.3854],
         [ 0.7050, -0.4590,  0.2962]],

        [[-0.3754, -2.4720, -2.1157],
         [ 0.3077, -1.9279, -0.5604]],

        [[ 0.1002, -1.2877, -2.3854],
         [ 0.5567,  0.0687, -1.5046]],

        [[ 0.2290, -2.1160, -1.1953],
         [ 0.3963, -0.6258, -1.1892]]])

In [21]:
attn_weights

tensor([[[1.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00],
         [1.3570e-03, 9.9864e-01, 0.0000e+00, 0.0000e+00],
         [9.9999e-01, 1.1649e-05, 4.2835e-11, 0.0000e+00],
         [1.6603e-01, 2.6574e-01, 5.6823e-01, 0.0000e+00]],

        [[1.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00],
         [6.5994e-01, 3.4006e-01, 0.0000e+00, 0.0000e+00],
         [3.2685e-05, 9.4755e-05, 9.9987e-01, 0.0000e+00],
         [8.1358e-02, 8.6933e-02, 6.6211e-01, 1.6960e-01]]])

#### torch mha

In [22]:
multihead_attn = nn.MultiheadAttention(embed_dim=embed_dim, num_heads=num_heads, bias=True)

In [23]:
# 手动设置模块的权重和偏置
with torch.no_grad():
    multihead_attn.in_proj_weight.copy_(in_proj_weight)
    multihead_attn.in_proj_bias.copy_(in_proj_bias)
    multihead_attn.out_proj.weight.copy_(W_o)
    multihead_attn.out_proj.bias.copy_(b_o)

In [24]:
# 调用 MultiheadAttention
attn_output_pytorch, attn_output_weights_pytorch = multihead_attn(
    query=query,
    key=key,
    value=value,
    attn_mask=src_mask,
    key_padding_mask=src_key_padding_mask
)

In [25]:
attn_output_pytorch

tensor([[[ 0.1002, -1.2877, -2.3854],
         [ 0.7050, -0.4590,  0.2962]],

        [[-0.3754, -2.4720, -2.1157],
         [ 0.3077, -1.9279, -0.5604]],

        [[ 0.1002, -1.2877, -2.3854],
         [ 0.5567,  0.0687, -1.5046]],

        [[ 0.2290, -2.1160, -1.1953],
         [ 0.3963, -0.6258, -1.1892]]], grad_fn=<ViewBackward0>)

In [26]:
attn_output_weights_pytorch

tensor([[[1.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00],
         [1.3570e-03, 9.9864e-01, 0.0000e+00, 0.0000e+00],
         [9.9999e-01, 1.1649e-05, 4.2835e-11, 0.0000e+00],
         [1.6603e-01, 2.6574e-01, 5.6823e-01, 0.0000e+00]],

        [[1.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00],
         [6.5994e-01, 3.4006e-01, 0.0000e+00, 0.0000e+00],
         [3.2685e-05, 9.4755e-05, 9.9987e-01, 0.0000e+00],
         [8.1358e-02, 8.6933e-02, 6.6211e-01, 1.6960e-01]]],
       grad_fn=<MeanBackward1>)